In [ ]:
from tqdm import tqdm
import re
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import random
import keras
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import IPython.display as display

# tfrecord에서 특정 부분 변경

In [ ]:
dataset = tf.data.TFRecordDataset('../data/face_data.tfrecord')
writer = tf.data.experimental.TFRecordWriter('../data/face_data3.tfrecord')
with tf.io.TFRecordWriter('../data/face_data3.tfrecord') as f:
    for i in dataset:
        feature_description = {"image": tf.io.VarLenFeature(dtype=tf.string),
                            "class": tf.io.VarLenFeature(dtype=tf.int64)}

        example = tf.io.parse_single_example(i, feature_description)
        image_raw = tf.sparse.to_dense(example["image"])[0]
        image = tf.io.decode_png(image_raw, channels=3)
        image = tf.image.resize(image, size=[224, 224])
        image = tf.io.encode_jpeg(tf.cast(image, dtype=tf.uint8)).numpy()    
        image_class = tf.sparse.to_dense(example["class"])
        # print(tf.cast(image_class, dtype=tf.int64))
        record = tf.train.Example(
                    features=tf.train.Features(
                        feature={
                            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                            'class': tf.train.Feature(int64_list=tf.train.Int64List(value=image_class.numpy())),
                            # 'bb_box': tf.train.Feature(float_list=tf.train.FloatList(value=bb_box_list)),
                            # 'landmark': tf.train.Feature(float_list=tf.train.FloatList(value=landmark_list))
                        }
                    )
                )
        f.write(record.SerializeToString())

# tfrecord 파일 합치기

In [ ]:
save_path = '../data/merge.tfrecord'
tfrecords_path = ['../data/face_data2.tfrecord', '../data/face_data3.tfrecord']
dataset = tf.data.TFRecordDataset(tfrecords_path)
writer = tf.data.experimental.TFRecordWriter(save_path)
writer.write(dataset)